<a href="https://colab.research.google.com/github/supertechnician/Chatbot-using-ANN/blob/main/Chatbot_Using_Multilayer_Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Chatbot based on Bag Of Words Model using Multilayer Perceptron
>

In [ ]:
%matplotlib inline
import os
import numpy as np
import random
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import nltk
import random
import json

## Install `optuna`

Optuna can be installed via `pip` or `conda`.

In [ ]:
!pip install --quiet optuna

In [ ]:
import optuna

optuna.__version__

'2.4.0'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
corpus_name = "Institution-bag of words"
corpus = os.path.join("/content/drive/My Drive/Colab Notebooks/data", corpus_name)

# NLTK Utilities

 

In [ ]:
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
print("HELLO")
def tokenize(sentence):
  return nltk.word_tokenize(sentence)

def stem(word):
  return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, words):
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1

    return bag

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
HELLO


 # Model
 

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

# Training

In [ ]:
# make it true to tune the hyperparameters using Optuna tuning method
tuning=False

with open(os.path.join(corpus, "intents.json"), 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

# Hyper-parameters
num_workers = 0
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples 

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=num_workers)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


# Train the model
def run_training( ):

    for epoch in range(num_epochs):
        for (words, labels) in train_loader:
            words = words.to(device)
            labels = labels.to(dtype=torch.long).to(device)

            # Forward pass
            outputs = model(words)
            # if y would be one-hot, we must apply
            # labels = torch.max(labels, 1)[1]
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if (epoch+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    print(f'final loss: {loss.item():.4f}')
    return loss.item()

# Train the model for tuning hyperparameters
def run_training_for_tuning(params):

    # Hyper-parameters
    num_workers = params["num_workers"]
    batch_size = params["batch_size"]
    learning_rate = params["learning_rate"]
    hidden_size = params["hidden_size"]

    for epoch in range(num_epochs):
        for (words, labels) in train_loader:
            words = words.to(device)
            labels = labels.to(dtype=torch.long).to(device)

            # Forward pass
            outputs = model(words)
            # if y would be one-hot, we must apply
            # labels = torch.max(labels, 1)[1]
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if (epoch+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    print(f'final loss: {loss.item():.4f}')
    return loss.item()


def objective(trial):
    params = {
        "num_workers": trial.suggest_int("num_workers", 0, 2),
        "hidden_size": trial.suggest_int("hidden_size", 300, 1200),
        "batch_size": trial.suggest_int("batch_size", 5, 10),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-6, 1e-3)
    }
    all_losses = []
    for f in range(2):
        temp_loss = run_training_for_tuning(params)
        all_losses.append(temp_loss)

    return np.mean(all_losses)


if tuning==True:
  if __name__ == "__main__":
      tuning = optuna.create_study(direction="minimize")
      tuning.optimize(objective, n_trials=3)

      print("best trial:")
      trial_ = tuning.best_trial

      print(trial_.values)
      print(trial_.params)

      scores = 0
      for j in range(1):
          scr = run_training(j, trial_.params, save_model=true)
          scores += scr
      print(scores)
else:
  run_training()

data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "tags": tags
}

# FILE = "data.pth"
FILE =os.path.join(corpus, "data.pth")
 
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

56 patterns
11 tags: ['College', 'Course', 'Department', 'Module', 'Student', 'Teacher', 'funny', 'goodbye', 'greeting', 'payments', 'thanks']
81 unique stemmed words: ["'s", 'a', 'accept', 'am', 'anyon', 'applic', 'are', 'belong', 'branch', 'bye', 'can', 'card', 'cash', 'centr', 'chatbot', 'colleg', 'cours', 'credit', 'day', 'degre', 'depart', 'do', 'doe', 'educ', 'enrol', 'for', 'funni', 'good', 'goodby', 'guid', 'ha', 'he', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'institu', 'instructor', 'is', 'joke', 'know', 'later', 'lectur', 'lot', 'made', 'mastercard', 'me', 'modul', 'name', 'of', 'onli', 'pay', 'paypal', 'professor', 'see', 'someth', 'stream', 'student', 'studi', 'subdivis', 'subject', 'take', 'task', 'teacher', 'tell', 'thank', 'that', 'the', 'there', 'thi', 'unit', 'univers', 'user', 'what', 'where', 'which', 'who', 'with', 'you']
81 11
Epoch [100/1000], Loss: 1.0251
Epoch [200/1000], Loss: 0.1961
Epoch [300/1000], Loss: 0.0102
Epoch [400/1000], Loss: 0.0042
Epoch [500/1000

# Chat


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open(os.path.join(corpus, "intents.json"), 'r') as json_data:
    intents = json.load(json_data)

# FILE = "data.pth"
FILE =os.path.join(corpus, "data.pth")
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "Guide"
print("Welcome to DMU! (type 'quit' to exit)"+"\n")
print("I am your guide"+"\n")
print("You can ask me questions about the details of the student."+"\n")
print("Type 'quit' to exit"+"\n")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}"+"\n")
    else:
        print(f"{bot_name}:Can you be more specific, please!"+"\n")

Welcome to DMU! (type 'quit' to exit)

I am your guide

You can ask me questions about the details of the student.

Type 'quit' to exit

You: Hi
Guide: Hi there, what can I do for you?

You: What is the name of the student?
Guide: Arpit Sharma is the name of the student

You: Who is the student?
Guide: The student's name is Arpit Sharma

You: student name?
Guide: The student's name is Arpit Sharma

You: name of the student?
Guide: Arpit Sharma

You: What is the name of the department?
Guide: The course belongs to the centre for Computational Intelligence

You: branch?
Guide: The course belongs to the CCI department

You: What is the name of the college?
Guide: He/She is a student at De Montfort University,Leicester

You: What is the name of the university?
Guide: He/She is a student at De Montfort University,Leicester

You: Which course is he studying?
Guide:  The student is studying intelligent systems and robotics, MSc

You: module name?
Guide: The student is studying NLP

You: Who i